## Evaluam performanta unei anumite epoci


In [1]:
import torch
from torch.utils.data import Dataset
from torch.nn import functional as F

%matplotlib widget

from utils.function_lib import *
from utils.LiverDataset import *
from utils.DiceLoss import *

from ModelBuilder.ResNet152_DeepLab import ResNet152DeepLab
from ModelBuilder.ResNeXt101_DeepLab import ResNeXt101DeepLab
from ModelBuilder.InceptionV3_DeepLab import InceptionV3DeepLab
from ModelBuilder.DenseNet201_DeepLab import DenseNet201DeepLab

## Transformari si dataset

In [2]:
transform_image = transforms.Compose(
    [
     # transforms.ToPILImage(),
     # transforms.Resize(256, interpolation=transforms.functional.InterpolationMode.BILINEAR, antialias=True),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.2], std=[0.2])
     ])
transform_mask = transforms.Compose(
    [
     # transforms.ToPILImage(),
     # transforms.Resize(512, interpolation=transforms.functional.InterpolationMode.NEAREST),
     transforms.ToTensor()
     ])

path = 'G:/MachineLearning/liver-database/'
model_path = 'G:/MachineLearning/models/ResNet152/'

dataset_test = LiverDataset(path + 'validation/', transform_image=transform_image, transform_mask=transform_mask)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=2, shuffle=True, num_workers=9)

## Incarcam modelul

In [3]:
# device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda')
model_name = 'ResNet152_200.pt'

model = torch.load(model_path + model_name)
model = model.to(device)
model = model.module

torch.cuda.empty_cache()
model = model.to(device)
loss_fn = DiceLoss().to(device)

## Evaluam datele de validare prin retea
Trecem prin retea toate imaginile din setul de validare/testare si calculam scorul Dice mediu pentru toate imaginile

In [4]:
loss = 0
model.eval()
with torch.no_grad():
    for X, Y in dataloader_test:
        pred = model(X.to(device))
        SoftMax = nn.Softmax2d()
        pred = SoftMax(pred).squeeze()
        pred = torch.nn.functional.interpolate(pred.unsqueeze(0), size=512, mode="bilinear", align_corners=False).squeeze()
        pred = torch.round(pred)

        # Y = Y.type(torch.long)
        Y = Y.to(device)
        loss += loss_fn(pred, Y).item()
loss = loss / len(dataloader_test)
print(f"Avg Dice: {loss:>8f} \n")

Avg Dice: 0.962403 

